In [ ]:
!pip install huggingface_hub --quiet
!pip install seqeval --quiet
!pip install datasets transformers==4.28.0 --quiet
!pip install wandb --quiet
!pip install evaluate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 13.8 MB/s eta 0:

In [ ]:
import datasets
import torch
from transformers import AdamW
import numpy as np
import wandb
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification, DataCollatorWithPadding
import gc
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts


from tqdm.auto import tqdm
from datasets import load_metric
import evaluate

from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification, DataCollatorWithPadding
from transformers import get_linear_schedule_with_warmup, get_inverse_sqrt_schedule, get_polynomial_decay_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup, get_cosine_schedule_with_warmup
from torch.optim import AdamW

In [ ]:
from huggingface_hub import login

access_read_token = '' # remember to add back in
access_write_token = '' # same here

login(token=access_read_token, add_to_git_credential=True)

Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
config = {
    'dataset': "",
    'model_name': "xlm-roberta-large-finetuned-conll03-german",
    # 'output_dir': '', # upload model later
    'lr': 1e-5,
    'seed': 316328460948097,
    'train/test': 0.1,
    'warmup': 3,
    'weight_decay': .13,
    'label_smoothing_factor': 0.17,
    'train_batch_size': 8,
    'eval_batch_size': 32,
    'epochs': 1000,
    'label_all_tokens': False,
    'do_checkpoint': True,
}

Modify Train/Test split for production

In [ ]:
# If the dataset is gated/private, make sure you have run huggingface-cli login
dataset = datasets.load_dataset(config['dataset'])
dataset = dataset['train'].train_test_split(test_size=config['train/test'], seed=config['seed'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 4500
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 500
    })
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config['model_name'], use_fast=True)

(…)-conll03-german/resolve/main/config.json:   0%|          | 0.00/886 [00:00<?, ?B/s]

(…)man/resolve/main/sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

(…)nll03-german/resolve/main/tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
example_text = dataset['train'][0]

tokenized_input = tokenizer(example_text['tokens'], is_split_into_words=True)

tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])

words_ids = tokenized_input.word_ids()

Fix Token correspondence disparity

1. Set -100 to labels for these special tokens at beginning and end of sentence
2. mask the subword representation after the first subword

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=config['label_all_tokens']):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
label_list = ['B-Modell', 'I-Modell', 'B-Marke', 'I-Marke', 'B-Produktlinie', 'I-Produktlinie', 'B-Schuhschaft-Typ', 'I-Schuhschaft-Typ',
 'B-Farbe', 'I-Farbe', 'B-US-Schuhgröße', 'I-US-Schuhgröße', 'B-EU-Schuhgröße', 'I-EU-Schuhgröße', 'B-No Tag', 'I-No Tag',
 'B-Akzente', 'I-Akzente', 'B-Abteilung', 'I-Abteilung', 'B-Produktart', 'I-Produktart', 'B-Muster', 'I-Muster',
 'B-Stil', 'I-Stil', 'B-Zwischensohlen-Typ', 'I-Zwischensohlen-Typ', 'B-Obermaterial', 'I-Obermaterial', 'B-Herstellernummer', 'I-Herstellernummer',
 'B-Gewebeart', 'I-Gewebeart', 'B-Verschluss', 'I-Verschluss', 'B-Erscheinungsjahr', 'I-Erscheinungsjahr', 'B-Anlass', 'I-Anlass',
 'B-Aktivität', 'I-Aktivität', 'B-Besonderheiten', 'I-Besonderheiten', 'B-Maßeinheit', 'I-Maßeinheit', 'B-Thema', 'I-Thema',
 'B-UK-Schuhgröße', 'I-UK-Schuhgröße', 'B-Dämpfungsgrad', 'I-Dämpfungsgrad', 'B-Herstellungsland und -region', 'I-Herstellungsland und -region',
 'B-Schuhweite', 'B-Jahreszeit', 'I-Jahreszeit', 'B-Laufsohlenmaterial', 'B-Innensohlenmaterial', 'I-Innensohlenmaterial', 'B-Obscure',
 'B-Futtermaterial', 'I-Futtermaterial', 'B-Charakter', 'I-Charakter', 'B-Stollentyp', 'B-Charakter Familie', 'I-Charakter Familie']

In [ ]:
id2label = {
    0: "B-Modell",
    1: "I-Modell",
    2: "B-Marke",
    3: "I-Marke",
    4: "B-Produktlinie",
    5: "I-Produktlinie",
    6: "B-Schuhschaft~Typ",
    7: "I-Schuhschaft~Typ",
    8: "B-Farbe",
    9: "I-Farbe",
    10: "B-US~Schuhgr\u00f6\u00dfe",
    11: "I-US~Schuhgr\u00f6\u00dfe",
    12: "B-EU~Schuhgr\u00f6\u00dfe",
    13: "I-EU~Schuhgr\u00f6\u00dfe",
    14: "B-No Tag",
    15: "I-No Tag",
    16: "B-Akzente",
    17: "I-Akzente",
    18: "B-Abteilung",
    19: "I-Abteilung",
    20: "B-Produktart",
    21: "I-Produktart",
    22: "B-Muster",
    23: "I-Muster",
    24: "B-Stil",
    25: "I-Stil",
    26: "B-Zwischensohlen~Typ",
    27: "I-Zwischensohlen~Typ",
    28: "B-Obermaterial",
    29: "I-Obermaterial",
    30: "B-Herstellernummer",
    31: "I-Herstellernummer",
    32: "B-Gewebeart",
    33: "I-Gewebeart",
    34: "B-Verschluss",
    35: "I-Verschluss",
    36: "B-Erscheinungsjahr",
    37: "I-Erscheinungsjahr",
    38: "B-Anlass",
    39: "I-Anlass",
    40: "B-Aktivit\u00e4t",
    41: "I-Aktivit\u00e4t",
    42: "B-Besonderheiten",
    43: "I-Besonderheiten",
    44: "B-Ma\u00dfeinheit",
    45: "I-Ma\u00dfeinheit",
    46: "B-Thema",
    47: "I-Thema",
    48: "B-UK~Schuhgr\u00f6\u00dfe",
    49: "I-UK~Schuhgr\u00f6\u00dfe",
    50: "B-D\u00e4mpfungsgrad",
    51: "I-D\u00e4mpfungsgrad",
    52: "B-Herstellungsland und ~region",
    53: "I-Herstellungsland und ~region",
    54: "B-Schuhweite",
    55: "B-Jahreszeit",
    56: "I-Jahreszeit",
    57: "B-Laufsohlenmaterial",
    58: "B-Innensohlenmaterial",
    59: "I-Innensohlenmaterial",
    60: "B-Obscure",
    61: "B-Futtermaterial",
    62: "I-Futtermaterial",
    63: "B-Charakter",
    64: "I-Charakter",
    65: "B-Stollentyp",
    66: "B-Charakter Familie",
    67: "I-Charakter Familie"
  }
label2id = {
    "B-Abteilung": 18,
    "B-Aktivit\u00e4t": 40,
    "B-Akzente": 16,
    "B-Anlass": 38,
    "B-Besonderheiten": 42,
    "B-Charakter": 63,
    "B-Charakter Familie": 66,
    "B-D\u00e4mpfungsgrad": 50,
    "B-EU~Schuhgr\u00f6\u00dfe": 12,
    "B-Erscheinungsjahr": 36,
    "B-Farbe": 8,
    "B-Futtermaterial": 61,
    "B-Gewebeart": 32,
    "B-Herstellernummer": 30,
    "B-Herstellungsland und ~region": 52,
    "B-Innensohlenmaterial": 58,
    "B-Jahreszeit": 55,
    "B-Laufsohlenmaterial": 57,
    "B-Marke": 2,
    "B-Ma\u00dfeinheit": 44,
    "B-Modell": 0,
    "B-Muster": 22,
    "B-No Tag": 14,
    "B-Obermaterial": 28,
    "B-Obscure": 60,
    "B-Produktart": 20,
    "B-Produktlinie": 4,
    "B-Schuhschaft~Typ": 6,
    "B-Schuhweite": 54,
    "B-Stil": 24,
    "B-Stollentyp": 65,
    "B-Thema": 46,
    "B-UK~Schuhgr\u00f6\u00dfe": 48,
    "B-US~Schuhgr\u00f6\u00dfe": 10,
    "B-Verschluss": 34,
    "B-Zwischensohlen~Typ": 26,
    "I-Abteilung": 19,
    "I-Aktivit\u00e4t": 41,
    "I-Akzente": 17,
    "I-Anlass": 39,
    "I-Besonderheiten": 43,
    "I-Charakter": 64,
    "I-Charakter Familie": 67,
    "I-D\u00e4mpfungsgrad": 51,
    "I-EU~Schuhgr\u00f6\u00dfe": 13,
    "I-Erscheinungsjahr": 37,
    "I-Farbe": 9,
    "I-Futtermaterial": 62,
    "I-Gewebeart": 33,
    "I-Herstellernummer": 31,
    "I-Herstellungsland und ~region": 53,
    "I-Innensohlenmaterial": 59,
    "I-Jahreszeit": 56,
    "I-Marke": 3,
    "I-Ma\u00dfeinheit": 45,
    "I-Modell": 1,
    "I-Muster": 23,
    "I-No Tag": 15,
    "I-Obermaterial": 29,
    "I-Produktart": 21,
    "I-Produktlinie": 5,
    "I-Schuhschaft~Typ": 7,
    "I-Stil": 25,
    "I-Thema": 47,
    "I-UK~Schuhgr\u00f6\u00dfe": 49,
    "I-US~Schuhgr\u00f6\u00dfe": 11,
    "I-Verschluss": 35,
    "I-Zwischensohlen~Typ": 27
  }

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(config['model_name'], num_labels=68, ignore_mismatched_sizes=True, id2label=id2label, label2id=label2id)

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large-finetuned-conll03-german and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([9, 1024]) in the checkpoint and torch.Size([68, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([68]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Data Collating for padding and Data Augmentation

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer, padding=True, return_tensors='pt')

In [ ]:
def compute_metrics(eval_preds):
  metric = datasets.load_metric('seqeval')
  pred_logits, labels = eval_preds

  predictions = [
      [id2label[eval_preds.item()] for (eval_preds, l) in zip(prediction, label) if l.item() != -100] for prediction, label in zip(pred_logits, labels)
  ]

  true_labels = [
      [id2label[l.item()] for (eval_preds, l) in zip(prediction, label) if l.item() != -100] for prediction, label in zip(pred_logits, labels)
  ]

  results = metric.compute(predictions=predictions, references=true_labels)

  return {
      "precision": results['overall_precision'],
      'recall': results['overall_recall'],
      'f1': results['overall_f1'],
      'accuracy': results['overall_accuracy']
  }

In [ ]:
# tokenized_dataset = tokenized_dataset.rename_column('id', 'input_ids')
tokenized_dataset = tokenized_dataset.with_format("torch", device=device)
tokenized_dataset.remove_columns(['id', 'tokens', 'ner_tags'])
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
train_dataloader = torch.utils.data.DataLoader(tokenized_dataset['train'], shuffle=True, batch_size=config['train_batch_size'], collate_fn=data_collator)
eval_dataloader = torch.utils.data.DataLoader(tokenized_dataset['test'], shuffle=False, batch_size=config['eval_batch_size'], collate_fn=data_collator)

In [ ]:
login(token=access_write_token, add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
wandb.login(key="") # remember to add this back in

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
num_epochs = config['epochs']

num_training_steps = num_epochs * len(train_dataloader)
warmup_training_steps = config['warmup'] * len(train_dataloader)

In [ ]:
optimizer = AdamW(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
criterion = torch.nn.CrossEntropyLoss(label_smoothing=config['label_smoothing_factor'])

# scheduler = get_linear_schedule_with_warmup(optimizer, warmup_training_steps, num_training_steps)
# scheduler = get_polynomial_decay_schedule_with_warmup(optimizer, warmup_training_steps, num_training_steps)
scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer, warmup_training_steps, num_training_steps, num_cycles=10)
# scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_training_steps, num_training_steps, num_cycles=0.5)
# scheduler = get_inverse_sqrt_schedule(optimizer, warmup_training_steps, num_training_steps)

# scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=50, T_mult=1, eta_min=1e-7)
# scheduler = ReduceLROnPlateau(optimizer, patience=3, mode='max', threshold=1e-4, factor=0.5)

In [ ]:
wandb.login(key="") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: Currently logged in as: jamesngai. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Create your wandb run
run = wandb.init(
    name = "rng-jesus", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project =  "eBay runs", # Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

In [ ]:
model = model.to(device)

In [ ]:
gc.collect() # These commands help you when you face CUDA OOM error
torch.cuda.empty_cache()

In [ ]:
f1_metric = evaluate.load("f1")

metric = datasets.load_metric('seqeval')

progress_bar = tqdm(range(num_training_steps))

val_f1 = 0

for epoch in range(num_epochs):
  model.train()
  for batch in train_dataloader:
    batch = {k: v.to(device) for k,v in batch.items()}
    outputs = model(**batch)
    loss = criterion(torch.transpose(outputs.logits, 1,2), batch['labels'])
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()
    progress_bar.update(1)
    scheduler.step()
    del batch

  model.eval()

  prediction_lst = []
  reference_lst = []

  for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
      outputs = model(**batch)

    logits = outputs.logits

    predictions = torch.argmax(logits, dim=-1)
    references = batch['labels']
    prediction_lst.extend(predictions.to('cpu'))
    reference_lst.extend(batch['labels'].to('cpu'))

  results = compute_metrics((prediction_lst, reference_lst))

  torch.cuda.empty_cache()
  print(results)

  f1_score = results['f1']
  f1_score_str = "{:.5f}".format(f1_score)

  curr_lr = float(optimizer.param_groups[0]['lr'])

  wandb.log({'validation_f1': f1_score, "learning_Rate": curr_lr})
  print(f"Epoch: {epoch}. f1: {f1_score_str}")
  print(f"Learning Rate: {curr_lr}")

  if f1_score > val_f1 and config['do_checkpoint']:
    model.save_pretrained(f"./checkpoint-best")
    tokenizer.save_pretrained(f"./checkpoint-best")
    print('Saving Best Model')


<ipython-input-26-11c7270be00f>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('seqeval')


  0%|          | 0/563000 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'precision': 0.6506524317912218, 'recall': 0.7166811846689896, 'f1': 0.6820725388601037, 'accuracy': 0.7234080717488789}
Epoch: 0. f1: 0.68207
Learning Rate: 3.3333333333333333e-06
Saving Best Model


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

{'precision': 0.9355119825708061, 'recall': 0.9351045296167247, 'f1': 0.9353082117185798, 'accuracy': 0.9422421524663677}
Epoch: 57. f1: 0.93531
Learning Rate: 4.19216228996966e-06
Saving Best Model


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'precision': 0.9413951454187623, 'recall': 0.9375, 'f1': 0.939443535188216, 'accuracy': 0.945829596412556}
Epoch: 58. f1: 0.93944
Learning Rate: 4.037106726649334e-06
Saving Best Model


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'precision': 0.9375136076638363, 'recall': 0.9377177700348432, 'f1': 0.9376156777354382, 'accuracy': 0.9443946188340807}
Epoch: 59. f1: 0.93762
Learning Rate: 3.883007149580867e-06
Saving Best Model
Buffered data was truncated after reaching the output size limit.